# Vector Database: Hands-on Tutorial

We will get a publicly available data, convert that data into chunks, embed the chunks, store them in vector database and retrieve the data with semantic search

#### [Signup to SingleStore](https://www.singlestore.com/cloud-trial/?utm_medium=referral&utm_source=pavan&utm_term=AV&utm_content=VECTRDB), get your free credits, try the tutorial using the Notebook feature

### Start with installing and importing the required libraries

In [9]:
!pip3 install wget --quiet
!pip3 install openai==1.3.3 --quiet
!pip3 install sentence-transformers --quiet

In [14]:
import json
import os
import pandas as pd
import wget

### Download the model

In [19]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('flax-sentence-embeddings/all_datasets_v3_mpnet-base')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Import data from the csv file (AG News is a subdataset of AG's corpus of news articles)

In [20]:
cvs_file_path = 'https://raw.githubusercontent.com/openai/openai-cookbook/main/examples/data/AG_news_samples.csv'
file_path = 'AG_news_samples.csv'

if not os.path.exists(file_path):
    wget.download(cvs_file_path, file_path)
    print('File downloaded successfully.')
else:
    print('File already exists in the local file system.')

df = pd.read_csv('AG_news_samples.csv')
df

File downloaded successfully.


,title,description,label_int,label
0,World Briefings,BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime M...,1,World
1,Nvidia Puts a Firewall on a Motherboard (PC Wo...,PC World - Upcoming chip set will include buil...,4,Sci/Tech
2,"Olympic joy in Greek, Chinese press",Newspapers in Greece reflect a mixture of exhi...,2,Sports
3,U2 Can iPod with Pictures,"SAN JOSE, Calif. -- Apple Computer (Quote, Cha...",4,Sci/Tech
4,The Dream Factory,"Any product, any shape, any size -- manufactur...",4,Sci/Tech
...,...,...,...,...
1995,You Control: iTunes puts control in OS X menu ...,MacCentral - You Software Inc. announced on Tu...,4,Sci/Tech
1996,Argentina beat Italy for place in football final,Favourites Argentina beat Italy 3-0 this morni...,2,Sports
1997,NCAA case no worry for Spurrier,Shortly after Steve Spurrier arrived at Florid...,2,Sports
1998,Secret Service Busts Cyber Gangs,The US Secret Service Thursday announced arres...,4,Sci/Tech


### You can see the data here

In [22]:
data = df.to_dict(orient='records')
data[0]

{'title': 'World Briefings',
 'description': 'BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the  quot;alarming quot; growth of greenhouse gases.',
 'label_int': 1,
 'label': 'World'}

### The next step is set up the database to store our data

In [28]:
%%sql

DROP TABLE IF EXISTS news_articles;
CREATE TABLE IF NOT EXISTS news_articles (
    title TEXT,
    description TEXT,
    genre TEXT,
    embedding BLOB,
    FULLTEXT(title, description)
);

++
||
++
++

### Get embeddings for every row based on the description column

In [29]:
descriptions = [row['description'] for row in data]
all_embeddings = model.encode(descriptions)
all_embeddings.shape

(2000, 768)

### Merge embedding values into data rows

In [36]:
for row, embedding in zip(data, all_embeddings):
    row['embedding'] = embedding

### Here is an example of one row of the combined data

In [37]:
data[0]

{'title': 'World Briefings',
 'description': 'BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the  quot;alarming quot; growth of greenhouse gases.',
 'label_int': 1,
 'label': 'World',
 'embedding': array([-1.42552713e-02, -1.03357071e-02,  1.25946105e-02,  8.40715785e-03,
        -6.92264410e-03, -8.77237227e-03, -5.38323671e-02,  1.95311196e-02,
         9.50564742e-02,  1.60899572e-02,  4.72200625e-02,  2.30231155e-02,
        -6.69442937e-02,  2.82599987e-03,  2.79738400e-02, -6.46088347e-02,
         5.52451760e-02, -4.02353071e-02, -2.22880822e-02, -1.65119395e-02,
         3.61824557e-02,  3.32110142e-03,  1.18329516e-02,  7.70277716e-03,
        -4.18954827e-02, -2.76368838e-02,  3.64982933e-02,  3.69321145e-02,
         5.97776957e-02,  8.05662386e-03,  3.38091105e-02, -1.52911590e-02,
         1.38111366e-02, -4.00905032e-03,  3.15332080e-08,  2.20

### Now, let’s populate the database with our data

In [45]:
%sql TRUNCATE TABLE news_articles;

import sqlalchemy as sa
from singlestoredb import create_engine

# Use create_table from singlestoredb since it uses the notebook connection URL
conn = create_engine().connect()

statement = sa.text('''
    INSERT INTO news_articles (
        title,
        description,
        genre,
        embedding
    )
    VALUES (
        :title,
        :description,
        :label,
        :embedding
    )
''')

conn.execute(statement, data)

### Let’s run semantic search, and get scores for the search term ‘India’

In [52]:
search_query = 'India'
search_embedding = model.encode(search_query)

query_statement = sa.text('''
    SELECT
        title,
        description,
        genre,
        DOT_PRODUCT(embedding, :embedding) AS score
    FROM news_articles
    ORDER BY score DESC
    LIMIT 10
''')

# Execute the SQL statement.
results = pd.DataFrame(conn.execute(query_statement, dict(embedding=search_embedding)))
print(results)

                                               title  \
0           Militants beat man thought to be from US   
1         Bomb at India Independence Parade Kills 15   
2        Microsoft Unveils Windows XP for India (AP)   
3  4 killed, 54 wounded in three separate attacks...   
4   Northeast Indian State Votes Amid Tight Security   
5  Why The Open-Source Model Can Work In India (T...   
6           Microsoft to Hire Hundreds More in India   
7  Bhopal victims commemorate 20th anniversary of...   
8  Follow-on shy Aussies lead India by 355 runs (...   
9                              No channel for series   

                                         description     genre     score  
0  HENDALA, Sri Lanka -- Day after day, locked in...     World  0.402497  
1  NEW DELHI - A bomb exploded during an Independ...     World  0.346617  
2  AP - Microsoft Corp. announced Wednesday that ...  Sci/Tech  0.344941  
3  Canadian Press - GAUHATI, India (AP) - Residen...     World  0.337517  
4   GUWA

### Now, let’s run a hybrid search to find articles about India.

In [57]:
hyb_query = 'Articles about India'
hyb_embedding = model.encode(hyb_query)

# Create the SQL statement.
hyb_statement = sa.text('''
    SELECT
        title,
        description,
        genre,
        DOT_PRODUCT(embedding, :embedding) AS semantic_score,
        MATCH(title, description) AGAINST (:query) AS keyword_score,
        (semantic_score + keyword_score) / 2 AS combined_score
    FROM news_articles
    ORDER BY combined_score DESC
    LIMIT 10
''')

# Execute the SQL statement.
hyb_results = pd.DataFrame(conn.execute(hyb_statement, dict(embedding=hyb_embedding, query=hyb_query)))
hyb_results


,title,description,genre,semantic_score,keyword_score,combined_score
0,Why The Open-Source Model Can Work In India (T...,TechWeb - An Indian Institute of Technology pr...,Sci/Tech,0.326729,0.380632,0.353681
1,Microsoft to Hire Hundreds More in India,"HYDERABAD, India (Reuters) - Microsoft Corp. ...",Business,0.229690,0.410682,0.320186
2,Home series defeats for India,"Australia, by winning the third Test at Nagpur...",Sports,0.165084,0.458400,0.311742
3,Microsoft Unveils Windows XP for India (AP),AP - Microsoft Corp. announced Wednesday that ...,Sci/Tech,0.228409,0.382128,0.305269
4,Putin favors veto right for India as permanent...,"In an apparent damage control exercise, Russia...",World,0.242365,0.353575,0.297970
5,Cricket: Aussies dominate India,Australia tighten their grip on the third Test...,World,0.066914,0.511894,0.289404
6,Southern Africa countries pledge enhanced trad...,"PORT LOUIS, Aug. 17 (Xinhuanet) -- Southern A...",World,0.181786,0.353575,0.267681
7,Follow-on shy Aussies lead India by 355 runs (...,AFP - Hosts India braced themselves for a harr...,World,0.150445,0.360399,0.255422
8,"54 dead, million flee homes as rains lash nort...",AFP - At least 54 people have died and more th...,World,0.159268,0.316645,0.237957
9,"Munabao-Khokhropar: Pakistan, India agree to r...",ISLAMABAD: Pakistan and India agreed on Friday...,World,0.154328,0.316645,0.235487
